In [2]:
import numpy as np
import imageio
import random
import os
import matplotlib.pyplot as plt
import glob

In [46]:
class randomWalk():
    '''
    Class to generate random walk on the lattice and save agent's path as a video.
    '''
    
    def __init__(self, gridSize=10, gridSizex = 10, gridSizey = 30, agentPositionID=255):
        '''
        Initializes the lattice and the agent.
        
        Parameters:
        gridSize - size of the square lattice, the default is 100
        gridSizex - horizontal size of the plane, the default is 100
        gridSizey - vertical size of the plane, the default is 100
        agentPositionID - assigned value to agent's start position and make it occupied,
            value should be an integer between 0 and 255, the default is 255 
        '''
        
        self.gridSize = gridSize
        self.gridSizex = gridSizex
        self.gridSizey = gridSizey
        self.agentPositionID = agentPositionID
        self.lattice = np.zeros((gridSize, gridSize), dtype = np.uint8)
        self.plane = np.zeros((gridSizex, gridSizey), dtype = np.uint8)
        self.agentPosition = (gridSize//2, gridSize//2)
        self.agentPositionAngleMoves = (gridSizex//2, gridSizey//2)
        self.lattice[self.agentPosition] = agentPositionID
        self.plane[self.agentPositionAngleMoves] = agentPositionID
        self.timeSteps = None
        self.moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.angleMoves = [(0, 1), (1, 1), (1, 0), (1, -1),
                          (0, -1), (-1, -1), (-1, 0), (-1, 1)]
        self.angles = [(270, 315), (315, 360), (0, 45), (45, 90),
                      (90, 135), (135, 180), (180, 225), (225, 270)]
        
    def agentRandomWalk(self, timeSteps = 1000):
        '''
        Iterates over every step and generates random walk of the agent and saves results as png file.

        Parameters:
        timeSteps - number of steps done by the agent, the default is 1000
        '''
        files = glob.glob(os.path.join('randomWalkFolder', '*')) # get access to all files in a folder
        for f in files:
            os.remove(f) # remove all files from folder before adding new ones
        self.timeSteps = timeSteps
        
        for t in range(timeSteps):
            moveInsideBoundaries = False
            while not moveInsideBoundaries:
                nextMove = random.choice(self.moves)
                nextPosition = (self.agentPosition[0] + nextMove[0],
                                self.agentPosition[1] + nextMove[1])
                if nextPosition[0] >= 0 and nextPosition[0] < self.gridSize and nextPosition[1] >= 0 and nextPosition[1] < self.gridSize:
                    moveInsideBoundaries = True
                    self.lattice[self.agentPosition] = 0
                    self.lattice[nextPosition] = self.agentPositionID
                    self.agentPosition = nextPosition
            file = os.path.join('randomWalkFolder', f'move_{t}.png')

            fig, ax = plt.subplots()
            ax.imshow(self.lattice, cmap='gray_r', vmin=0, vmax=255) # shows pictures on a plot, cmap for white background, v is a gray scale
            ax.set_xticks(np.arange(0, self.gridSize, self.gridSize//10))
            ax.set_yticks(np.arange(0, self.gridSize, self.gridSize//10))
            ax.invert_yaxis()
            ax.grid(color = 'gray', linestyle = '--', linewidth = 0.25)
            plt.title("Agent's random walk on a square latter")
            plt.savefig(file, facecolor='white')
            plt.close()
            
    def agentRandomWalkWithAngles(self, timeSteps = 100):
        '''
        Iterates over every step and generates random walkf of the agent on the plane with randomly chosen angles
        and saves results as png file.
        
        Parameters:
        timeSteps - number of steps done by the agent, the default is 1000
        '''
        files = glob.glob(os.path.join('randomWalkWithAnglesFolder', '*')) # get access to all files in a folder
        for f in files:
            os.remove(f) # remove all files from folder before adding new ones
        self.timeSteps = timeSteps
        
        for t in range(timeSteps):
            moveInsideBoundaries = False
            while not moveInsideBoundaries:
                angle = np.random.uniform(0.0, 360.0)
                for i in range(len(self.angles)):
                    if angle >= self.angles[i][0] and angle < self.angles[i][1]:
                        nextPosition = (self.agentPositionAngleMoves[0] + self.angleMoves[i][0],
                                        self.agentPositionAngleMoves[1] + self.angleMoves[i][1])
                        if nextPosition[0] >= 0 and nextPosition[0] < self.gridSizex and nextPosition[1] >= 0 and nextPosition[1] < self.gridSizey:
                            moveInsideBoundaries = True
                            self.plane[self.agentPositionAngleMoves] = 0
                            self.plane[nextPosition] = self.agentPositionID
                            self.agentPositionAngleMoves = nextPosition
            file = os.path.join('randomWalkWithAnglesFolder', f'Move_{t}.png')
            
            fig, ax = plt.subplots()
            ax.imshow(self.plane, cmap='gray_r', vmin=0, vmax=255) # shows pictures on a plot, cmap for white background, v is a gray scale
            #ax.set_xticks(np.arange(0, self.gridSizex, self.gridSizex//10))
            #ax.set_yticks(np.arange(0, self.gridSizey, self.gridSizey//10))
            ax.invert_yaxis()
            ax.grid(color = 'gray', linestyle = '--', linewidth = 0.25)
            plt.title("Agent's random walk on a plane")
            plt.savefig(file, facecolor='white')
            plt.close()
    
    def createRandomWalkMovie(self, walkType):
        '''
        Creates a movie from saved images of a random walk of the agent.
        
        Parameters:
        walkType - type of walk which we want to create a move from, we can choose from
        latticeWalk, anglesWalk, graphWalk
        '''
        if walkType == 'latticeWalk':
            filename = 'AgentRandomWalk.mp4'
            folder = 'randomWalkFolder'
        elif walkType == 'anglesWalk':
            filename = 'AgentRandomWalkWithAngles.mp4'
            folder = 'randomWalkWithAnglesFolder'
            
        with imageio.get_writer(filename, mode = 'I', fps = 10) as w: # mode I means that function uses Images to create video
            for i in range(self.timeSteps):     
                file = os.path.join(folder, f'move_{i}.png')
                img = imageio.imread(file)
                w.append_data(img)

In [47]:
walkAngle = randomWalk(gridSizex = 20, gridSizey = 40)

In [48]:
walkAngle.agentRandomWalkWithAngles()

In [49]:
walkAngle.createRandomWalkMovie(walkType = 'anglesWalk')